# Table of Contents

1. &nbsp; [Introduction](#1-Introduction)
2. &nbsp; [Preamble](#2-Preamble)
3. &nbsp; [Data](#3-Data)
4. &nbsp; [Visualization](#4-Visualization)
5. &nbsp; [Baseline](#5-Baseline:-Naive-Bayes)
6. &nbsp; [On Cross Validation](#6-On-Cross-Validation)
7. &nbsp; [Models](#7-Models)
8. &nbsp; [Conclusion](#8-Conclusion)


# 1 Introduction

This notebook is a simple starter analysis for the Iris dataset.

Questions and feedback are welcome!

### Background

Some helpful links about the *Iris* family of flowers:

- [Flower Anatomy](http://www.northernontarioflora.ca/flower_term.cfm) from the Northern Ontario Plant Database
- [*Iris setosa*](https://www.wildflower.org/plants/result.php?id_plant=IRSE),
[*Iris versicolor*](https://www.wildflower.org/plants/result.php?id_plant=IRVE2),
and
[*Iris virginica*](https://www.wildflower.org/plants/result.php?id_plant=IRVI)
from the Lady Bird Johnson Wildflower Center


### License

My work is licensed under CC0:

- Overview: https://creativecommons.org/publicdomain/zero/1.0/
- Legal code: https://creativecommons.org/publicdomain/zero/1.0/legalcode.txt

All other rights remain with their respective owners.

# 2 Preamble

The usual `python` preamble:

- `jupyter` magic
- `numpy`
- `pandas`
- `seaborn` + `matplotlib`
- `scikit-learn`

Click the `Code` button to take a look at hidden cells.

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier,
    AdaBoostClassifier, ExtraTreesClassifier)
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

plt.rcParams['figure.figsize'] = (19,5)
sns.set(style='whitegrid', color_codes=True, font_scale=1.5)
np.set_printoptions(suppress = True, linewidth = 200)
pd.set_option('display.max_rows', 100)

In [ ]:
from typing import Sequence

def plot_confmat(
        y: pd.Series,
        y_hat: Sequence,
        rotate_x: int = 0,
        rotate_y: int = 'vertical') \
        -> None:
    """
    Plot confusion matrix using `seaborn`.
    """
    classes = y.unique()
    ax = sns.heatmap(
        confusion_matrix(y, y_hat),
        xticklabels=classes,
        yticklabels=classes,
        annot=True,
        square=True,
        cmap="Blues",
        fmt='d',
        cbar=False)
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    plt.xticks(rotation=rotate_x)
    plt.yticks(rotation=rotate_y, va='center')
    plt.xlabel('Predicted Value')
    plt.ylabel('True Value')
    
def seq(start, stop, step=None) -> np.ndarray:
    """
    Inclusive sequence.
    """
    if step is None:
        if start < stop:
            step = 1
        else:
            step = -1

    if is_int(start) and is_int(step):
        dtype = 'int'
    else:
        dtype = None

    d = max(n_dec(step), n_dec(start))
    n_step = np.floor(round(stop - start, d + 1) / step) + 1
    delta = np.arange(n_step) * step
    return np.round(start + delta, decimals=d).astype(dtype)

def is_int(x) -> bool:
    """
    Whether `x` is int.
    """
    return isinstance(x, (int, np.integer))

def n_dec(x) -> int:
    """
    Number of decimal places, using `str` conversion.
    """
    if x == 0:
        return 0
    _, _, dec = str(x).partition('.')
    return len(dec)


Let's store the path to our data and set a global seed for reproducible results, then get started.

In [ ]:
csv = '../input/Iris.csv'
seed = 0

# 3 Data

We'll load the `csv` using `pandas` and take a look at some summary statistics.

## Peek

Let's load the first 10 rows of our dataset and make a todo list.

In [ ]:
pd.read_csv(csv, nrows=10)

### To do

- Drop the `Id` column.
- Convert `Species` to `category`.
- Split the `DataFrame` into `X` and `y` (predictors and target feature, respectively).
- Remove `Iris-` from `Species` labels.  (This is to shrink our confusion matrix.)

Note that:

- `Species` is our target.
- All of our features are floats, which simplifies preprocessing.

## Load Full Dataset

Let's load our data (do the todos) and take a look at `X` and `y`.

In [ ]:
def load_iris(csv, y='Species'):
    df = pd.read_csv(
        csv,
        usecols=lambda x: x != 'Id',
        dtype={y: 'category'},
        engine='c',
    )
    X = df.drop(columns=y)
    y = df[y].map(lambda s: s.replace('Iris-', ''))
    return X, y

In [ ]:
X, y = load_iris(csv)

## Predictors

In [ ]:
X.info()

### Key takeaways

- No missing data.
- 4 columns.
- Our features are 2 dimensional: sepals and petals, so polynomial features (ie,  area) might help.
- This is a very small dataset (5 KB), which gives us the luxury of fast model training.

## Target

In [ ]:
y.value_counts()

We have a balanced dataset:
- 3 species
- 50 rows each

## Summary

- Goal: classify flower species
- 150 observations
- 4 predictors, all floats
- 3 balanced target categories
- No missing data

Next, let's take a closer look at our data.

# 4 Visualization

We'll use `seaborn` for 2 plots:

- A `pairplot` to see pairwise relationships.
- A `boxplot` for feature scaling.

## Pair Plot

In [ ]:
def pair(X, y):
    sns.pairplot(pd.concat((X, y), axis=1, copy=False), hue=y.name, diag_kind='kde', size=2.2)

In [ ]:
pair(X, y)

Pairwise comparisons separate our target quite well, especially *Iris setosa* (blue).

Let's move on to the boxplot.

## Boxplot

In [ ]:
def box(X):
    plt.figure(figsize=(10,5))
    sns.boxplot(data=X, orient='v');

In [ ]:
box(X)

This view of our features is useful for feature scaling.  Notably:

- All features use the same scale: cm.
- Therefore, all features should be strictly positive.
- Features occupy different value ranges (eg, sepal length: 4 to 8 cm, petal width: 0 to 3 cm).
- And, features have different variances (eg, petal length vs sepal width).

Overall, we probably don't need to worry too much about feature scaling for this dataset.  Standardization (*z*-score scaling) should be fine.

Next, let's train a simple baseline model.

# 5 Baseline: Naive Bayes

The choice of model is arbitrary, but we want something convenient as our baseline.  So, let's use Gaussian naive Bayes.  It's simple and fast, and it works out of the box with no preprocessing and no tuning.

In [ ]:
def baseline(X, y):
    acc = GaussianNB().fit(X, y).score(X, y).round(4) * 100
    print(f'{acc}% accuracy')

In [ ]:
baseline(X, y)

Great.  Our baseline is 96% accuracy.  Let's break down our error rate using a confusion matrix.

In [ ]:
def confuse(X, y):
    plt.figure(figsize=(4.2,4.2))
    model = GaussianNB().fit(X, y)
    plot_confmat(y, model.predict(X))

In [ ]:
confuse(X, y)

As expected *Iris setosa* is easy to classify.  On the other hand, we mistake both *versicolor* and *virginica* for the other.

Let's explore the performance of our baseline in the next section.

# 6 On Cross Validation

The holy grail of machine learning is generalization.  We want to know how well our model performs on data it hasn't seen before.  Kaggle competitions use a hidden holdout set (aka the *test set*) to uniformly rank submissions, but we don't have that here.  So, let's use cross validation to simulate a holdout set.  The method is simple:

1. Split the data into `train` and `test`.
2. Fit the model on `train`.
3. Measure accuracy on `test`.
4. Repeat to taste.

## How should we split our data?

- First, use a fixed seed for reproducible results.
- Second, cross validation is often performed using the *k*-fold method, but we'll be using `sklearn`'s `StratifiedShuffleSplit` instead.  This gives us better control over training size, which is important for the next question.

## Where should we split our data?

This is somewhat arbitrary, so let's try a few different options.

- We'll use 10%, 20%, ..., 90% of our total data as `train`, and the remainder will be `test`.
- We'll split our data 1000 times for each percentage.

## To do

- Gaussian naive Bayes classifier
- 9 percentages, 10% to 90%
- 1000 splits for each percentage
- 9000 models total

## Results

In [ ]:
def baseline_cv(X, y, model, n_splits, fractions):
    history = np.empty((n_splits, len(fractions)))
    
    for i, fr in enumerate(fractions):
        shuffle = StratifiedShuffleSplit(n_splits, train_size=fr, test_size=None, random_state=seed)
        for j, (idx_tr, idx_te) in enumerate(shuffle.split(X, y)):
            tr_X, tr_y = X.iloc[idx_tr], y.iloc[idx_tr]
            te_X, te_y = X.iloc[idx_te], y.iloc[idx_te]
            history[j,i] = model.fit(tr_X, tr_y).score(te_X, te_y)
    
    df = pd.DataFrame(history, columns=[f'{int(fr*150)}' for fr in fractions])
    
    plt.figure(figsize=(16,7))
    sns.boxplot(data=df)
    plt.xlabel('Train Size')
    plt.ylabel('Accuracy Score')
    plt.title('Accuracy vs Training Size')

In [ ]:
%%time
baseline_cv(X, y, GaussianNB(), n_splits=1000, fractions=seq(0.1, 0.9, step=0.1))

Key takeaways:

- Our baseline model performs well across the board, starting at just 30 observations (20% of our data), with around 95% accuracy.
- At `train` size 15, accuracy degrades quite a bit if we get unlucky (down to 65%), but overall, model accuracy is very consistent.

### Note

As `train` grows, `test` *shrinks*, because the split is complementary, and thus accuracy becomes less granular:

- At `train` size 30, each misclassification reduces accuracy by less than 1%.
- At `train` size 135, the marginal reduction is over 6%.

This is why variance increases left to right, above `train` size 75.  Such a comparison is *not* apples to apples.  The moral of the story is that `test` size matters.

## The Plan

Let's use 45 observations as `train` (stratified 15/15/15).  The remaining 105 observations will be `test`.  Hopefully this strikes a good balance between predictive power and generalization, but it's a fairly arbitrary choice.

### Aside

We won't be doing any model tuning or stacking in this notebook.  We won't be splitting `train` at all, so our workflow will differ quite a bit from the Kaggle competition setup, which often splits data into at least 4 parts:

1. Private leaderboard (final ranking)
2. Public leaderboard (rate limited cross validation)
3. Train/dev (unlimited cross validation)
4. Train/train (true `train` data)

# 7 Models

## Preprocessing

For `X`, let's add some polynomial features and use standardization.

In [ ]:
X_pipeline = make_pipeline(
    PolynomialFeatures(interaction_only=True, include_bias=False),
    StandardScaler(),
)


We'll leave `y` as is.

In [ ]:
y_pipeline = None

In [ ]:
def preprocess(X, test_X, y, test_y, X_pipeline, y_pipeline, n_train, seed):
    if X_pipeline:
        X = X_pipeline.fit_transform(X)
        test_X = X_pipeline.transform(test_X)

    if y_pipeline:
        y = y_pipeline.fit_transform(y)
        test_y = y_pipeline.transform(y)
    
    return X, test_X, y, test_y

## Which Models

Here are the models we'll be using:

In [ ]:
model_dict = {
    'RandomForest': RandomForestClassifier(random_state=seed),
    'NaiveBayes': GaussianNB(),
    'kNN': KNeighborsClassifier(),
    'ExtraTrees': ExtraTreesClassifier(random_state=seed),
    'GradientBoost': GradientBoostingClassifier(random_state=seed),
    'Bagg': BaggingClassifier(random_state=seed),
    'AdaBoost': AdaBoostClassifier(random_state=seed),
    'GaussianProc': GaussianProcessClassifier(random_state=seed),
    'Logistic': LogisticRegression(random_state=seed),
}

Just the default parameters.

## Metrics

Thus far, we've been using accuracy, but there's no official leaderboard metric.  So, let's also take a look at log loss.

### The Plan

- `train` size = 45
- `test` size = 105 (complement)
- 1000 random splits for each model
- Use the same seed

In [ ]:
def cv(model_dict, X, y, X_pipeline, y_pipeline, n_train, n_splits, seed):
    acc_dict = dict()
    loss_dict = dict()
    for k, m in model_dict.items():
        acc = []
        loss = []
        shuffle = StratifiedShuffleSplit(n_splits,
                                         train_size=n_train,
                                         test_size=None,
                                         random_state=seed)
        for idx_train, idx_test in shuffle.split(X, y):
            tr_X, te_X, tr_y, te_y = preprocess(X.iloc[idx_train], X.iloc[idx_test],
                                                y.iloc[idx_train], y.iloc[idx_test],
                                                X_pipeline, y_pipeline, n_train, seed)
            m.fit(tr_X, tr_y)

            y_hat_acc = m.predict(te_X)
            acc += [accuracy_score(te_y, y_hat_acc)]

            y_hat_loss = m.predict_proba(te_X)
            loss += [log_loss(te_y, y_hat_loss)]

        acc_dict[k] = acc
        loss_dict[k] = loss

    return pd.DataFrame(acc_dict), pd.DataFrame(loss_dict)

In [ ]:
%%time
acc, loss = cv(model_dict, X, y, X_pipeline, y_pipeline, n_train=45, n_splits=1000, seed=seed)

In [ ]:
print('Accuracy (%): standard deviation')
print((np.std(acc).round(4)*100).to_string())

In [ ]:
print('Log loss: standard deviation')
print((np.std(loss).round(2)).to_string())

## Metrics Plots

Let's take a look.

In [ ]:
def plot_metrics(acc, loss):
    plt.figure(figsize=(10,14))
    plt.subplots_adjust(hspace=0.6)
    
    plt.subplot(2,1,1)
    plt.title('Accuracy')
    sns.boxplot(data=acc*100)
    plt.ylim(80,100)
    plt.xticks(rotation='vertical')

    plt.subplot(2,1,2)
    plt.title('Log Loss (lower is better)')
    sns.boxplot(data=loss)
    plt.xticks(rotation='vertical')

In [ ]:
plot_metrics(acc, loss)

## Winners and Losers

We shouldn't read too much into model performance without having done any optimization, but this is a good starting point.  A few things that might be worth exploring:

- Gaussian processes and logistic regression have very stable log loss.
- Extra trees performs well.
- Accuracy is more or less the same across the board.  Logistic regression stands out, but that might be due to, eg lack of regularization.
- Log loss is much more varied, but most models do overlap.  Gaussian processes is a bit high, though.
- Both plots have quite a few outliers.

Overall, the log loss is difficult to interpret, but 95% accuracy (+/- 2%) seems like a solid baseline for this dataset.

# 8 Conclusion

## What's Missing

- Train/test split: We used a 45/105 split.  Try different splits--30/120, 60/90, or 90/60--and see what happens.
- Error analysis: Take a look at the errors.  Is a particular flower hard to classify?  Does each model misclassify the same flowers?
- Algorithm anslysis: Use the bootstrap--or maybe something else--to better understand the bias/variance characteristics of each model.
- Optimization: Model parameters, hyperparameters, cross validation, regularization, stacking, etc.
- Feature engineering/analysis: Go beyond polynomial features--what works and what doesn't, and why?
- Logs: Saving/loading data, and tracking everything we have and haven't tried--a detailed journal of our thought process.
- Metrics: What we're optimizing should influence how we optimize.
- More models: `xgboost` and `pytorch` or `keras`.

## Final Words

There are surely mistakes and better ways to go about things.  If you have any ideas, I would appreciate your feedback.

That's all for this notebook.  I hope you found something useful.